# Artificial Neural Network 2

In [ ]:
#Importando librearias
import sys
import os
import json
import mlflow
import mlflow.keras
import tempfile
import pandas as pd
import seaborn as sns
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
sys.path.append('../')
sys.path.append('../../')
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import kerastuner as kt
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
import Python.Private.constants as constants
from Python.Style.styles import  *
from scipy.stats import expon, randint
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score, confusion_matrix

In [ ]:
#Desactivando wrnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Leyendo datos
training = spark.sql("SELECT * FROM training")
training_us = spark.sql("SELECT * FROM training_us")
training_os = spark.sql("SELECT * FROM training_os")
testing1 = spark.sql("SELECT * FROM testing1")
testing2 = spark.sql("SELECT * FROM testing2")

In [ ]:
#To delete columns
total_columns = training.columns
tdc = ['serialNumber','serialNumber_neighbor','fixed_path','FE-Comments','Atenna','Conjunto','PSNumber']
training_cols = list(filter(lambda x: x not in tdc, total_columns))
#Dividiendo
train_t, train_v = training.randomSplit([0.8,0.2], seed = 1010)
train_us_t, train_us_v = training_us.randomSplit([0.8,0.2], seed = 1010)
train_os_t, train_os_v = training_os.randomSplit([0.8,0.2], seed = 1010)

In [ ]:
#Seleccionando columnas
train_t = train_t.select(*training_cols)
train_v = train_v.select(*training_cols)
train_us_t = train_us_t.select(*training_cols)
train_us_v = train_us_v.select(*training_cols)
train_os_t = train_os_t.select(*training_cols)
train_os_v = train_os_v.select(*training_cols)
#Conjuntos de datos de prueba
testing1 = testing1.select(*training_cols)
testing2 = testing2.select(*training_cols)


In [ ]:
#Dividiendo datos training
train_t_y = train_t.select("Communicating")
train_t_x = train_t.drop("Communicating")
train_v_y = train_v.select("Communicating")
train_v_x = train_v.drop("Communicating")
#Dividiendo datos training undersampled
train_us_t_y = train_us_t.select("Communicating")
train_us_t_x = train_us_t.drop("Communicating")
train_us_v_y = train_us_v.select("Communicating")
train_us_v_x = train_us_v.drop("Communicating")
#Dividiendo datos training oversampled
train_os_t_y = train_os_t.select("Communicating")
train_os_t_x = train_os_t.drop("Communicating")
train_os_v_y = train_os_v.select("Communicating")
train_os_v_x = train_os_v.drop("Communicating")
#Dividiendo conjunto de datos de testeo
testing1_y = testing1.select("Communicating")
testing1_x = testing1.drop("Communicating")
testing2_y = testing2.select("Communicating")
testing2_x = testing2.drop("Communicating")

In [ ]:
#Conversion for General Training Data
train_t_y = train_t_y.toPandas().astype(np.float32).values
train_t_x = train_t_x.toPandas().astype(np.float32).values
train_v_y = train_v_y.toPandas().astype(np.float32).values
train_v_x = train_v_x.toPandas().astype(np.float32).values
#Conversion for Training Data - Undersampled
train_us_t_y = train_us_t_y.toPandas().astype(np.float32).values
train_us_t_x = train_us_t_x.toPandas().astype(np.float32).values
train_us_v_y = train_us_v_y.toPandas().astype(np.float32).values
train_us_v_x = train_us_v_x.toPandas().astype(np.float32).values
#Conversion for Training Data - Oversampled
train_os_t_y = train_os_t_y.toPandas().astype(np.float32).values
train_os_t_x = train_os_t_x.toPandas().astype(np.float32).values
train_os_v_y = train_os_v_y.toPandas().astype(np.float32).values
train_os_v_x = train_os_v_x.toPandas().astype(np.float32).values
#Conversión conjuntos de datos de testeo
testing1_y = testing1_y.toPandas().astype(np.float32).values
testing1_x = testing1_x.toPandas().astype(np.float32).values
testing2_y = testing2_y.toPandas().astype(np.float32).values
testing2_x = testing2_x.toPandas().astype(np.float32).values

In [ ]:
class MyHyperModel(HyperModel):
    def build(self, hp):
        inputs = Input(shape=(train_t_x.shape[1],))

        # Primera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_1', min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_1', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(inputs)
        x = Dropout(hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.1))(x)

        # Segunda capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)

        # Capa de salida
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='binary_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
#Tuner
tuner = Hyperband(
    MyHyperModel(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='dl3_dir',
    project_name='keras_tuner_hyperband'
)

Reloading Tuner from dl3_dir/keras_tuner_hyperband/tuner0.json


In [ ]:
mlflow.set_experiment('/Users/victor.hugo.borrayo@gmail.com/DL2')

## Ajustando para undersampled

In [ ]:
#Umbrales de discretización
def find_threshold(model, x, y):
    continuous_pred = model.predict(x)
    thres = []
    f1_macro = []
    accuracy = []
    mpca = []
    for threshold in np.arange(0.1,1.0,0.01):
        thres.append(threshold)
        pred = (continuous_pred > threshold).astype(int)
        f1_macro.append(f1_score(y, pred, average='macro'))
        accuracy.append(accuracy_score(y, pred))
        cm = confusion_matrix(y, pred)
        #Balanced accuracy
        tp = cm[1,1]
        fp = cm[0,1]
        tn = cm[0,0]
        fn = cm[1,0]
        #Precisión por clase
        precision_class_0 = tn / (tn + fp)
        precision_class_1 = tp / (tp + fn)
        temp_mpca = (precision_class_0 + precision_class_1) / 2
        mpca.append(temp_mpca)
    #Encontrando máximos
    tres_f1 = thres[f1_macro.index(max(f1_macro))]
    tres_acc = thres[accuracy.index(max(accuracy))]
    thres_mpca = thres[mpca.index(max(mpca))]
    thresholds = {
        "f1_thres": tres_f1,
        "acc_thres": tres_acc,
        "mpca_thres": thres_mpca
    }
    return thresholds


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_training_us"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_undersampled")
    #Almacenando información de distribución
    tuner.search(
        train_us_t_x, train_us_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_us_v_x, train_us_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_us_v_x,train_us_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='macro')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='macro')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})    


2024/06/17 01:49:57 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/17 01:49:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmps8usz3v6/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmps8usz3v6/model/data/model/assets


323/323 [==============================] - 0s 1ms/step


In [ ]:
tuner.results_summary()

Results summary
Results in dl3_dir/keras_tuner_hyperband
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0023 summary
Hyperparameters:
units_1: 352
l2_1: 0.0003307813841875074
dropout_1: 0.1
units_2: 256
l2_2: 0.0002750302510155711
dropout_2: 0.2
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8467804789543152

Trial 0020 summary
Hyperparameters:
units_1: 416
l2_1: 0.0005578816564959981
dropout_1: 0.1
units_2: 64
l2_2: 2.2497906903236934e-05
dropout_2: 0.4
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0016
Score: 0.8328161239624023

Trial 0012 summary
Hyperparameters:
units_1: 480
l2_1: 0.00019964078033611233
dropout_1: 0.0
units_2: 64
l2_2: 1.4367632995710257e-05
dropout_2: 0.0
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0011
Score: 0.8302947878837585

Trial 0016 summary
Hyper

## Ajustando para oversampled

In [ ]:
#Tuner
tuner2 = Hyperband(
    MyHyperModel(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='dl3_dir',
    project_name='keras_tuner_hyperband_os'
)

Reloading Tuner from dl3_dir/keras_tuner_hyperband_os/tuner0.json


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_training_os_1"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner2.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner2.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner2.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_os_v_x,train_os_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='macro')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='macro')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})  

2024/06/17 01:57:46 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/17 01:57:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmpmtelc56d/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmpmtelc56d/model/data/model/assets


7664/7664 [==============================] - 7s 936us/step


In [ ]:
tuner2.results_summary()

Results summary
Results in dl3_dir/keras_tuner_hyperband_os
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0024 summary
Hyperparameters:
units_1: 512
l2_1: 3.4371369161912445e-05
dropout_1: 0.1
units_2: 32
l2_2: 0.0035855145517498283
dropout_2: 0.4
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0018
Score: 0.9548739194869995

Trial 0016 summary
Hyperparameters:
units_1: 384
l2_1: 9.00421736859498e-05
dropout_1: 0.30000000000000004
units_2: 160
l2_2: 0.0006816981385010221
dropout_2: 0.1
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.9504045248031616

Trial 0026 summary
Hyperparameters:
units_1: 224
l2_1: 0.0005015976228726456
dropout_1: 0.1
units_2: 64
l2_2: 1.5737132764906787e-05
dropout_2: 0.0
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9487488865852356

Tria

## Ajustando para oversampled 2

In [ ]:
class MyHyperModel2(HyperModel):
    def build(self, hp):
        inputs = Input(shape=(train_t_x.shape[1],))

        # Primera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_1', min_value=32, max_value=1024, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_1', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(inputs)
        x = Dropout(hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.1))(x)

        # Segunda capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)

        # Tercera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)


        # Capa de salida
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='binary_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
#Tuner
tuner3 = Hyperband(
    MyHyperModel2(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='dl3_dir',
    project_name='keras_tuner_hyperband_dl_os_2'
)

Reloading Tuner from dl3_dir/keras_tuner_hyperband_dl_os_2/tuner0.json


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_training_os_2"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner3.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner3.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner3.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_os_v_x,train_os_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='macro')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='macro')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/17 02:01:14 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/17 02:01:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmpweutp55w/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-45ebb-fca2d-fa624/tmpweutp55w/model/data/model/assets


7664/7664 [==============================] - 8s 1ms/step


In [ ]:
tuner3.results_summary()

Results summary
Results in dl3_dir/keras_tuner_hyperband_dl_os_2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0016 summary
Hyperparameters:
units_1: 640
l2_1: 0.00020780367938768153
dropout_1: 0.1
units_2: 64
l2_2: 6.224065081134312e-05
dropout_2: 0.0
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.9711406826972961

Trial 0024 summary
Hyperparameters:
units_1: 1024
l2_1: 1.5121543909399543e-05
dropout_1: 0.1
units_2: 64
l2_2: 0.0064305919014236886
dropout_2: 0.1
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.9647016525268555

Trial 0017 summary
Hyperparameters:
units_1: 160
l2_1: 1.3759719874419433e-05
dropout_1: 0.1
units_2: 160
l2_2: 7.498738290228463e-05
dropout_2: 0.0
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.9584135413